# 2024-10

In [1]:
import aocd, setup

# Get Puzzle on this Year and Day.
year = 2024
day = 10
session_id = setup.get_sessionid()  # current session id

puzzle = aocd.get_puzzle(session_id, year=year, day=day)            # get puzzle info
data = aocd.get_data(session_id, year=year, day=day).splitlines()   # get input data


In [2]:
# Review the data and examples.
print("Input length:", len(data))
print("Input sample:")

for i in range(7):
    print(data[i])

eg_data = puzzle.examples[0].input_data
print("\nInput example:")
print(eg_data)

eg_answer_a = puzzle.examples[0].answer_a
print("answer a:", eg_answer_a)
eg_answer_b = puzzle.examples[0].answer_b
print("answer b:", eg_answer_b)
eg_extra = puzzle.examples[0].extra
print("extra:", eg_extra)


Input length: 45
Input sample:
012589824304345454345653403898787654310365890
323456712215216783238732312345693201223456781
450144603304306690129641002105892100378901672
343223454456987601234553213456743898465432543
234019765567898501246764789122156784567643456
105678876678983454334895601033021093298932167
987619989569332569543210592344534580112341098

Input example:
89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732
answer a: 36
answer b: None
extra: None


# Part 1

In [27]:
import numpy as np

In [267]:
# Process INPUT data.
input = []
for i in range(len(data)):          # Individualise each digit. 
    input.append(list(data[i]))

map = np.array(input)             # Convert to np.array.
map = map.astype(np.int32)      # Convert to int32.
map.shape

(45, 45)

In [29]:
# Process SAMPLE DATA.
input = eg_data.splitlines()
for i in range(len(input)):
    input[i] = list(input[i])

map = np.array(input)         # Change to np.array.
map = map.astype(np.int32)  # Change type to int32.
map

array([[8, 9, 0, 1, 0, 1, 2, 3],
       [7, 8, 1, 2, 1, 8, 7, 4],
       [8, 7, 4, 3, 0, 9, 6, 5],
       [9, 6, 5, 4, 9, 8, 7, 4],
       [4, 5, 6, 7, 8, 9, 0, 3],
       [3, 2, 0, 1, 9, 0, 1, 2],
       [0, 1, 3, 2, 9, 8, 0, 1],
       [1, 0, 4, 5, 6, 7, 3, 2]], dtype=int32)

In [222]:
# Find starting indices in a map.
def get_starting_indices(map):
    start_idx = np.where(map == 0)                                     # get grid indices for peak = 0.
    start_idx = np.reshape(np.ravel(start_idx, order='F'), (-1,2))     # ravel and reshape the coordinates to get [[x, y]].
    return start_idx


In [259]:
# Climb the map recursively. Return the coordinates of the peak = 9. 
def climb(map, this_row, this_col, peaks):

    # print(map[this_row][this_col], '\t[', this_row, ', ', this_col, ']\t', len(peaks), '\t', peaks)

    if (map[this_row, this_col] == 9):                  # If reached the peak, termininate recursion. 
        return np.array([this_row, this_col])
        
    directions = [[-1, 0], [0, 1], [1, 0], [0, -1]]     # NESW order of direction.

    for r, c in directions:

        if ((this_row + r >= 0) & (this_row + r < len(map))        # Check for map boundaries - row.
            & (this_col + c >= 0) & (this_col + c < len(map[0]))):     # check for map boundaries - col.

            # Condition needed to be separated or it will hit an index out of bounds error.
            if (map[this_row][this_col] +1 == map[this_row + r][this_col + c]):     # Check that height is 1 greater than this location.

                result = climb(map, this_row + r, this_col + c, peaks)      # Get coordinates of '9': recursive results.
                peaks = np.append(peaks, result)                            # Add coordinates to 'peaks' array.
                peaks = np.unique(np.reshape(peaks, (-1, 2)), axis = 0)     # Continually reshape and only keep unique peaks.

    return peaks
            


In [233]:
get_starting_indices(map)

array([[0, 2],
       [0, 4],
       [2, 4],
       [4, 6],
       [5, 2],
       [5, 5],
       [6, 0],
       [6, 6],
       [7, 1]])

In [270]:
# Find the sum of the scores of all trailheads.
start_idxs = get_starting_indices(map)
answer_a = 0

for r, c in start_idxs:
    p = climb(map, r, c, np.array([]))
    answer_a += len(p)

print(answer_a)


538


In [271]:
response = aocd.submit(answer_a, part=1, day=day, year=year, session=session_id, reopen=False)

current_day is only available in December (EST)


That's the right answer!  You are one gold star closer to finding the Chief Historian. [Continue to Part Two]


# Part 2

In [ ]:
# response = aocd.submit(answer_b, part=2, day=day, year=year, session=session_id, reopen=False)